In [8]:
this_notebook_name = "UltrasoundBoneSegmentation-TF2"

# Update these folder names for your computer
# Contains all output
output_data_folder = r"E:\Perk\Data"
# Contains the training data
images_data_folder = r"E:\Perk\Dataset\Sub-evaluation-6-Erasmus-MC\Sub-evaluation-6-Erasmus-MC\train\images"
labels_data_folder = r"E:\Perk\Dataset\Sub-evaluation-6-Erasmus-MC\Sub-evaluation-6-Erasmus-MC\train\labels"

overwrite_existing_data_files = False

# All results and output will be archived with this timestamp

import datetime
save_timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
print("Save timestamp: {}".format(save_timestamp))

# Learning parameters

import numpy as np

ultrasound_size = 128
num_classes = 2
num_epochs = 500
batch_size = 128
max_learning_rate = 0.02
min_learning_rate = 0.00001
regularization_rate = 0.0001
filter_multiplier = 8
class_weights = np.array([0.1, 0.9])
learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

# Training data augmentation parameters

max_shift_factor = 0.12
max_rotation_angle = 10
max_zoom_factor = 1.1
min_zoom_factor = 0.8

# Evaluation parameters

acceptable_margin_mm = 1.0
mm_per_pixel = 1.0

roc_thresholds = [0.9, 0.8, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1,
                  0.08, 0.06, 0.04, 0.02, 0.01,
                  0.008, 0.006, 0.004, 0.002, 0.001]

'''
Provide NxM numpy array to schedule cross validation
N rounds of validation will be performed, leaving out M patients in each for validation data
All values should be valid patient IDs, or negative. Negative values are ignored.

Example 1: a leave-one-out cross validation with 3 patients would look like this:
validation_schedule_patient = np.array([[0],[1],[2]])

Example 2: a leave-two-out cross validation on 10 patients would look like this:
validation_schedule_patient = np.array([[0,1],[2,3],[4,5],[6,7],[8,9]])

Example 3: leave-one-out cross validation with 3 patients, then training on all available data (no validation):
validation_schedule_patient = np.array([[0],[1],[2],[-1]])
'''

# The two variables below replace the previous variable validation_schedule_patient
# because the new data is not organized by patient

# functions as the dimension M in validation_schedule_patient
percent_val_split = 20
# functions as the dimension N in validation_schedule_patient
num_validation_rounds = 1

# Uncomment for faster debugging
roc_thresholds = [0.8, 0.6, 0.4, 0.2, 0.1, 0.01, 0.001]
num_epochs = 5

Save timestamp: 2020-07-16_11-35-07


In [3]:
import os
import sys
from random import sample
from pathlib import Path
from PIL import Image
from scipy.ndimage import zoom

from ipywidgets import IntProgress
from IPython.display import display, HTML

import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf

import ultrasound_batch_generator as generator
import evaluation_metrics

In [23]:
# Import aigt modules

parent_folder = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(parent_folder)

import Models.segmentation_unet as unet
import utils

In [10]:
# Creating standard folders to save data and logs

data_arrays_fullpath, notebooks_save_fullpath, results_save_fullpath, models_save_fullpath, val_data_fullpath =\
    utils.create_standard_project_folders(output_data_folder)

In [16]:
# loads data, resizes it, and adds it to arrays

ultrasound_array,segmentation_array = [],[]
for root, dirs, files in os.walk(images_data_folder):
    for file in files:
        if file.endswith('.png'):
            image_path=os.path.join(root, file)
            arr = np.array(Image.open(image_path))
            
            shape = arr.shape
            x_axis, y_axis = shape[0], shape[1]
            arr=zoom(arr,(128/x_axis,128/y_axis))
            arr = arr[:,:,np.newaxis]
            arr = arr[np.newaxis,:,:,:]
            
            ultrasound_array.append(arr)

for root, dirs, files in os.walk(labels_data_folder):
    for file in files:
        if file.endswith('.png'):
            label_path=os.path.join(root, file)
            arr = np.array(Image.open(label_path))
            
            shape = arr.shape
            x_axis, y_axis = shape[0], shape[1]
            arr=zoom(arr,(128/x_axis,128/y_axis))
            arr = arr[:,:,np.newaxis]
            arr = arr[np.newaxis,:,:,:]
            
            segmentation_array.append(arr)

# This dimension isn't passed into the network, 
# It is just a way to organize all the arrays into one large array
n_images = np.shape(ultrasound_array)[0]

print(n_images)
print(np.shape(ultrasound_array))
print(np.shape(segmentation_array))

81
(81, 1, 128, 128, 1)
(81, 1, 128, 128, 1)


In [14]:
# Prepare validation rounds

# This situation would represent there is not enough data
# to perform the desired split and number of validation rounds
if percent_val_split*num_validation_rounds>100:
    raise Exception("Percent split or number of rounds are too high")

print("Planning {} round(s) of validation".format(num_validation_rounds))

Planning 1 round(s) of validation


In [22]:
# Print training parameters, to archive them together with the notebook output.

time_sequence_start = datetime.datetime.now()

print("Timestamp for saved files: {}".format(save_timestamp))
print("\nTraining parameters")
print("Number of epochs:    {}".format(num_epochs))
print("Step size maximum:   {}".format(max_learning_rate))
print("Step size decay:     {}".format(learning_rate_decay))
print("Batch size:          {}".format(batch_size))
print("Regularization rate: {}".format(regularization_rate))
print("")
print("Saving validation predictions in: {}".format(val_data_fullpath))
print("Saving models in:                 {}".format(models_save_fullpath))

# ROC data will be saved in these containers

val_best_metrics    = dict()
val_fuzzy_metrics   = dict()
val_aurocs          = np.zeros(num_validation_rounds)
val_best_thresholds = np.zeros(num_validation_rounds)

# Perform validation rounds

for val_round_index in range(num_validation_rounds):
    
    # Prepare data arrays
    
    train_ultrasound_data = np.zeros(
        [0,
         np.shape(ultrasound_array)[2],
         np.shape(ultrasound_array)[3],
         np.shape(ultrasound_array)[4]])
    
    train_segmentation_data = np.zeros(
        [0,
         np.shape(segmentation_array)[2],
         np.shape(segmentation_array)[3],
         np.shape(segmentation_array)[4]])
    
    val_ultrasound_data = train_ultrasound_data
    val_segmentation_data = val_ultrasound_data
    
    print(np.shape(val_ultrasound_data))
    print(np.shape(train_ultrasound_data))
    print(np.shape(val_segmentation_data))
    print(np.shape(train_segmentation_data))
    
    for image_index in range(n_images):
        
        # if the image at [image_index] falls into the range
        # designated by the variable percent_val_split
        if (val_round_index)*n_images*(percent_val_split/100) <= image_index < (val_round_index+1)*n_images*(percent_val_split/100):
            #print("added image {} to validation split".format(image_index))
            val_ultrasound_data = np.concatenate((val_ultrasound_data,
                                                    ultrasound_array[image_index]))
            val_segmentation_data = np.concatenate((val_segmentation_data,
                                                      segmentation_array[image_index]))
            
        else:
            #print("added image {} to training split".format(image_index))
            train_ultrasound_data = np.concatenate((train_ultrasound_data,
                                                    ultrasound_array[image_index]))
            train_segmentation_data = np.concatenate((train_segmentation_data,
                                                      segmentation_array[image_index]))
    
    n_train = np.shape(train_ultrasound_data)[0]
    n_val = np.shape(val_ultrasound_data)[0]
    
    print(np.shape(val_ultrasound_data))
    print(np.shape(train_ultrasound_data))
    print(np.shape(val_segmentation_data))
    print(np.shape(train_segmentation_data))
    
    print("\n*** Leave-one-out round # {}".format(val_round_index))
    print("    Training on {} images, validating on {} images...".format(n_train, n_val))
    
    val_segmentation_data_onehot = tf.keras.utils.to_categorical(val_segmentation_data, num_classes)
    
    # Create and train model
    
    model = unet.segmentation_unet(ultrasound_size, num_classes, filter_multiplier, regularization_rate)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=max_learning_rate, decay=learning_rate_decay),
        loss=unet.weighted_categorical_crossentropy(class_weights),
        metrics=["accuracy"]
    )
    
    # model.summary()
        
    training_generator = generator.UltrasoundSegmentationBatchGenerator(
        train_ultrasound_data,
        train_segmentation_data[:, :, :, 0],
        batch_size,
        (ultrasound_size, ultrasound_size),
        max_shift_factor=max_shift_factor,
        min_zoom_factor=min_zoom_factor,
        max_zoom_factor=max_zoom_factor,
        max_rotation_angle=max_rotation_angle
    )
        
    training_time_start = datetime.datetime.now()
    
    if n_val > 0:
        training_log = model.fit_generator(
            training_generator,
            validation_data=(val_ultrasound_data, val_segmentation_data_onehot),
            epochs=num_epochs,
            verbose=0)
    else:
        training_log = model.fit_generator(training_generator, epochs=num_epochs, verbose=0)
    
    training_time_stop = datetime.datetime.now()
    
    # Pring training log
    
    print("  Training time: {}".format(training_time_stop-training_time_start))
    
    # Plot training loss and metrics
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    
    axes[0].plot(training_log.history['loss'], 'bo--')
    if n_val > 0:
        axes[0].plot(training_log.history['val_loss'], 'ro-')
    axes[0].set(xlabel='Epochs (n)', ylabel='Loss')
    if n_val > 0:
        axes[0].legend(['Training loss', 'Validation loss'])
    
    axes[1].plot(training_log.history['accuracy'], 'bo--')
    if n_val > 0:
        axes[1].plot(training_log.history['val_accuracy'], 'ro-')
    axes[1].set(xlabel='Epochs (n)', ylabel='Accuracy')
    if n_val > 0:
        axes[1].legend(['Training accuracy', 'Validation accuracy'])
    
    fig.tight_layout()
    
    # Archive trained model with unique filename based on notebook name and timestamp
    
    model_file_name = this_notebook_name + "_model-" + str(val_round_index) + "_" + save_timestamp + ".h5"
    model_fullname = os.path.join(models_save_fullpath, model_file_name)
    model.save(model_fullname)
    
    # Predict on validation data
    
    if n_val > 0:
        y_pred_val  = model.predict(val_ultrasound_data)

        # Saving predictions for further evaluation

        val_prediction_filename = save_timestamp + "_prediction_" + str(val_round_index) + ".npy"
        val_prediction_fullname = os.path.join(val_data_fullpath, val_prediction_filename)
        np.save(val_prediction_fullname, y_pred_val)

        # Validation results

        vali_metrics_dicts, vali_best_threshold_index, vali_area = evaluation_metrics.compute_roc(
            roc_thresholds, y_pred_val, val_segmentation_data, acceptable_margin_mm, mm_per_pixel)

        val_fuzzy_metrics[val_round_index] = evaluation_metrics.compute_evaluation_metrics(
            y_pred_val, val_segmentation_data, acceptable_margin_mm, mm_per_pixel)

        val_best_metrics[val_round_index]    = vali_metrics_dicts[vali_best_threshold_index]
        val_aurocs[val_round_index]          = vali_area
        val_best_thresholds[val_round_index] = roc_thresholds[vali_best_threshold_index]
    
    # Printing total time of this validation round
    
    print("\nTotal round time:  {}".format(datetime.datetime.now() - training_time_start))
    print("")


time_sequence_stop = datetime.datetime.now()

print("\nTotal training time:   {}".format(time_sequence_stop - time_sequence_start))

Timestamp for saved files: 2020-07-16_11-35-07

Training parameters
Number of epochs:    5
Step size maximum:   0.02
Step size decay:     3.998e-05
Batch size:          128
Regularization rate: 0.0001

Saving validation predictions in: E:\Perk\Data\PredictionsValidation
Saving models in:                 E:\Perk\Data\SavedModels
(0, 128, 128, 1)
(0, 128, 128, 1)
(0, 128, 128, 1)
(0, 128, 128, 1)
(17, 128, 128, 1)
(64, 128, 128, 1)
(17, 128, 128, 1)
(64, 128, 128, 1)

*** Leave-one-out round # 0
    Training on 64 images, validating on 17 images...


IndexError: index 2 is out of bounds for axis 1 with size 2

In [ ]:
# Arrange results in tables

metric_labels = [
    "AUROC",
    "best thresh",
    "best TP",
    "best FP",
    "best recall",
    "best precis",
    "fuzzy recall",
    "fuzzy precis",
    "fuzzy Fscore"
]

results_labels = []

for label in metric_labels:
    results_labels.append("Vali " + label)

results_df = pd.DataFrame(columns = results_labels)

for i in range(num_validation_rounds):
    if i in val_best_metrics.keys():
        results_df.loc[i] = [
            val_aurocs[i],
            val_best_thresholds[i],
            val_best_metrics[i][evaluation_metrics.TRUE_POSITIVE_RATE],
            val_best_metrics[i][evaluation_metrics.FALSE_POSITIVE_RATE],
            val_best_metrics[i][evaluation_metrics.RECALL],
            val_best_metrics[i][evaluation_metrics.PRECISION],
            val_fuzzy_metrics[i][evaluation_metrics.RECALL],
            val_fuzzy_metrics[i][evaluation_metrics.PRECISION],
            val_fuzzy_metrics[i][evaluation_metrics.FSCORE]
        ]

display(results_df)

print("\nAverages")

results_means_df = results_df.mean()
display(results_means_df)

In [ ]:
# Print the last ROC curve for visual verification that we catch the optimal point

n = len(roc_thresholds)

roc_x = np.zeros(n)
roc_y = np.zeros(n)

for i in range(n):
    roc_x[i] = vali_metrics_dicts[i][evaluation_metrics.FALSE_POSITIVE_RATE]
    roc_y[i] = vali_metrics_dicts[i][evaluation_metrics.SENSITIVITY]
    # print("Threshold = {0:4.2f}  False pos rate = {1:4.2f}  Sensitivity = {2:4.2f}"
    #       .format(roc_thresholds[i], roc_x[i], roc_y[i]))

    
plt.figure()
plt.ylim(-0.01, 1.01)
plt.xlim(-0.01, 1.01)
plt.plot(roc_x, roc_y, color='darkred', lw=2)

In [ ]:
# Save results table

csv_filename = this_notebook_name + "_" + save_timestamp + ".csv"
csv_fullname = os.path.join(results_save_fullpath, csv_filename)
results_df.to_csv(csv_fullname)

print("Results saved to: {}".format(csv_fullname))

In [ ]:
# Display sample results

num_vali = val_ultrasound_data.shape[0]
num_show = 3
if num_vali < num_show:
    num_show = 0
num_col = 4
    
indices = [i for i in range(num_vali)]
sample_indices = sample(indices, num_show)
sample_indices = [105, 195, 391]

threshold = 0.5

# Uncomment for comparing the same images
# sample_indices = [105, 195, 391, 133, 142]

fig = plt.figure(figsize=(18, num_show*5))
for i in range(num_show):
    a0 = fig.add_subplot(num_show, num_col, i*num_col+1)
    img0 = a0.imshow(np.flipud(val_ultrasound_data[sample_indices[i], :, :, 0].astype(np.float32)))
    a0.set_title("Ultrasound #{}".format(sample_indices[i]))
    a1 = fig.add_subplot(num_show, num_col, i*num_col+2)
    img1 = a1.imshow(np.flipud(val_segmentation_data_onehot[sample_indices[i], :, :, 1]), vmin=0.0, vmax=1.0)
    a1.set_title("Segmentation #{}".format(sample_indices[i]))
    c = fig.colorbar(img1, fraction=0.046, pad=0.04)
    a2 = fig.add_subplot(num_show, num_col, i*num_col+3)
    img2 = a2.imshow(np.flipud(y_pred_val[sample_indices[i], :, :, 1]), vmin=0.0, vmax=1.0)
    a2.set_title("Prediction #{}".format(sample_indices[i]))
    c = fig.colorbar(img2, fraction=0.046, pad=0.04)
    a3 = fig.add_subplot(num_show, num_col, i*num_col+4)
    img3 = a3.imshow((np.flipud(y_pred_val[sample_indices[i], :, :, 1]) > threshold), vmin=0.0, vmax=1.0)
    c = fig.colorbar(img3, fraction=0.046, pad=0.04)
    a3.set_title("Thresholded #{}".format(sample_indices[i]))

In [ ]:
# Save notebook so all output is archived by the next cell

from IPython.display import Javascript
script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
Javascript(script)

In [ ]:
# Export HTML copy of this notebook

notebook_file_name = this_notebook_name + "_" + save_timestamp + ".html"
notebook_fullname = os.path.join(notebooks_save_fullpath, notebook_file_name)

os.system("jupyter nbconvert --to html " + this_notebook_name + " --output " + notebook_fullname)
print("Notebook saved to: {}".format(notebook_fullname))